In [ ]:
from sklearn.neighbors import KNeighborsClassifier # классификатор
from sklearn.neighbors import KNeighborsRegressor # регрессор
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
import numpy as np
from sklearn.linear_model import LinearRegression

In [ ]:
df = pd.read_csv("Price_euros_train.csv")
df

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
1,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
2,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60
3,6,Acer,Aspire 3,Notebook,15.6,1366x768,AMD A9-Series 9420 3GHz,4GB,500GB HDD,AMD Radeon R5,Windows 10,2.1kg,400.00
4,7,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.2GHz,16GB,256GB Flash Storage,Intel Iris Pro Graphics,Mac OS X,2.04kg,2139.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,1315,Asus,X556UJ-XO044T (i7-6500U/4GB/500GB/GeForce,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,4GB,500GB HDD,Nvidia GeForce 920M,Windows 10,2.2kg,720.32
1104,1316,Lenovo,Yoga 500-14ISK,2 in 1 Convertible,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 6500U 2.5GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 10,1.8kg,638.00
1105,1318,Lenovo,IdeaPad 100S-14IBR,Notebook,14.0,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2GB,64GB Flash Storage,Intel HD Graphics,Windows 10,1.5kg,229.00
1106,1319,HP,15-AC110nv (i7-6500U/6GB/1TB/Radeon,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,6GB,1TB HDD,AMD Radeon R5 M330,Windows 10,2.19kg,764.00


In [ ]:
df.isna().sum()

laptop_ID            0
Company              0
Product              0
TypeName             0
Inches               0
ScreenResolution    11
Cpu                  0
Ram                  0
Memory               0
Gpu                  0
OpSys               53
Weight              22
Price_euros          0
dtype: int64

#Убераем NAN 

#Разрешение

In [ ]:
df['ScreenResolution'].value_counts()

Full HD 1920x1080                                429
1366x768                                         241
IPS Panel Full HD 1920x1080                      191
IPS Panel Full HD / Touchscreen 1920x1080         46
Full HD / Touchscreen 1920x1080                   36
1600x900                                          19
Quad HD+ / Touchscreen 3200x1800                  15
Touchscreen 1366x768                              12
IPS Panel 4K Ultra HD 3840x2160                    9
4K Ultra HD / Touchscreen 3840x2160                9
IPS Panel 4K Ultra HD / Touchscreen 3840x2160      8
Touchscreen 2560x1440                              7
4K Ultra HD 3840x2160                              6
IPS Panel 1366x768                                 6
Touchscreen 2256x1504                              5
IPS Panel Retina Display 2304x1440                 5
IPS Panel Retina Display 2560x1600                 5
IPS Panel Touchscreen 2560x1440                    5
IPS Panel Touchscreen 1920x1200               

In [ ]:
df['ScreenResolution'] = df['ScreenResolution'].apply(lambda x: "Full HD 1920x1080" if pd.isna(x) else x)

In [ ]:
df['ScreenResolution']

0                                        1440x900
1              IPS Panel Retina Display 2880x1800
2              IPS Panel Retina Display 2560x1600
3                                        1366x768
4              IPS Panel Retina Display 2880x1800
                          ...                    
1103                                     1366x768
1104    IPS Panel Full HD / Touchscreen 1920x1080
1105                                     1366x768
1106                                     1366x768
1107                                     1366x768
Name: ScreenResolution, Length: 1108, dtype: object

#Операционная система

In [ ]:
indexes = df['OpSys'].isna()

In [ ]:
df.loc[indexes, 'OpSys'] = df.loc[indexes, 'Company'].apply(lambda x: 'macOS' if x == 'Apple' else 'Windows 10')
df[indexes]

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
8,13,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.8GHz,16GB,256GB SSD,AMD Radeon Pro 555,macOS,1.83kg,2439.97
10,15,Apple,"MacBook 12""",Ultrabook,12.0,IPS Panel Retina Display 2304x1440,Intel Core M m3 1.2GHz,8GB,256GB SSD,Intel HD Graphics 615,macOS,0.92kg,1262.40
30,38,Dell,Inspiron 5770,Notebook,17.3,IPS Panel Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,128GB SSD + 1TB HDD,AMD Radeon 530,Windows 10,2.8kg,979.00
32,40,HP,ProBook 450,Notebook,15.6,Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,Nvidia GeForce 930MX,Windows 10,2.1kg,879.00
39,48,Asus,Rog Strix,Gaming,17.3,Full HD 1920x1080,AMD Ryzen 1700 3GHz,8GB,256GB SSD + 1TB HDD,AMD Radeon RX 580,Windows 10,3.2kg,1299.00
53,63,Lenovo,IdeaPad 520S-14IKB,Notebook,14.0,IPS Panel Full HD 1920x1080,Intel Core i3 7130U 2.7GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.7kg,599.00
56,67,Asus,ZenBook Pro,Ultrabook,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,512GB SSD,Nvidia GeForce GTX 1050 Ti,Windows 10,1.8kg,1983.00
110,129,HP,Probook 440,Notebook,14.0,Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,1TB HDD,Intel UHD Graphics 620,Windows 10,1.63kg,812.00
142,169,MSI,GE63VR 7RE,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1060,Windows 10,2.4kg,1799.00
148,177,Toshiba,Satellite Pro,Notebook,15.6,1366x768,Intel Core i3 7100U 2.4GHz,4GB,500GB HDD,Intel HD Graphics 620,Windows 10,2kg,489.00


#Вес

In [ ]:
indexes2 = df['Weight'].isna()
indexes1 = [not i for i in indexes2]

lr = LinearRegression()
lr.fit(df.loc[indexes1][['Inches']], df.loc[indexes1]['Weight'].apply(lambda x: float(x[:-2])))

LinearRegression()

In [ ]:
df.loc[indexes2, 'Weight'] = [str(i) + 'kg' for i in lr.predict(df.loc[indexes2][['Inches']])]
df[indexes2]

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
13,18,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.9GHz,16GB,512GB SSD,AMD Radeon Pro 560,macOS,2.181862899274864kg,2858.00
25,32,Asus,E402WA-GA010T (E2-6110/2GB/32GB/W10),Notebook,14.0,1366x768,AMD E-Series E2-6110 1.5GHz,2GB,32GB Flash Storage,AMD Radeon R2,Windows 10,1.6387447752128983kg,199.00
37,46,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.0GHz,8GB,256GB SSD,Intel Iris Graphics 540,macOS,1.3671857131819154kg,1419.00
51,61,Dell,Inspiron 5770,Notebook,17.3,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,16GB,256GB SSD + 2TB HDD,AMD Radeon 530,Windows 10,2.9189517819303896kg,1299.00
61,73,Dell,Inspiron 5370,Ultrabook,13.3,IPS Panel Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8GB,256GB SSD,AMD Radeon 530,Windows 10,1.3671857131819154kg,955.00
180,216,Asus,ROG GL703VD-GC028T,Gaming,17.3,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1050,Windows 10,2.9189517819303896kg,1407.00
192,229,Dell,Alienware 17,Gaming,17.3,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1060,Windows 10,2.9189517819303896kg,2456.34
206,245,Lenovo,IdeaPad 320-15ISK,Notebook,15.6,1366x768,Intel Core i3 6006U 2GHz,8GB,128GB SSD,Intel HD Graphics 520,Windows 10,2.259451202712288kg,589.00
228,269,Dell,Inspiron 3567,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,1TB HDD,AMD Radeon R5 M430,Windows 10,2.259451202712288kg,565.00
253,295,Acer,Chromebook C910-C2ST,Notebook,15.6,1366x768,Intel Celeron Dual Core 3205U 1.5GHz,2GB,16GB SSD,Intel HD Graphics,Chrome OS,2.259451202712288kg,199.00


#Обрабатываем данные

In [ ]:
for col in df.select_dtypes(['object']).columns:
  print(col, len(df[col].value_counts()))

Company 18
Product 548
TypeName 6
ScreenResolution 36
Cpu 112
Ram 9
Memory 37
Gpu 106
OpSys 9
Weight 176


In [ ]:
df = df.drop(columns=['Product', 'laptop_ID'])

In [ ]:
T = 'Flash'

def memory(x):
  for item in x.split('+'):
    if len(item.split()) == 3:
      s, t, _ = item.split()
      if t == T: 
        if s[-2] == 'T':
          return float(s[:-2]) * 1024 
        else:
          return float(s[:-2]) 
    else:
      s, t = item.split()
      if t == T: 
        if s[-2] == 'T':
          return float(s[:-2]) * 1024 
        else:
          return float(s[:-2]) 
  return 0


df['Flash Storage'] = df['Memory'].apply(memory)
T = 'SSD'
df['SSD'] = df['Memory'].apply(memory)
T = 'HDD'
df['HDD'] = df['Memory'].apply(memory)
df = df.drop(columns=['Memory'])
df

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Gpu,OpSys,Weight,Price_euros,Flash Storage,SSD,HDD
0,Apple,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,Intel HD Graphics 6000,macOS,1.34kg,898.94,128.0,0.0,0.0
1,Apple,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,AMD Radeon Pro 455,macOS,1.83kg,2537.45,0.0,512.0,0.0
2,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60,0.0,256.0,0.0
3,Acer,Notebook,15.6,1366x768,AMD A9-Series 9420 3GHz,4GB,AMD Radeon R5,Windows 10,2.1kg,400.00,0.0,0.0,500.0
4,Apple,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.2GHz,16GB,Intel Iris Pro Graphics,Mac OS X,2.04kg,2139.97,256.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,Asus,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,4GB,Nvidia GeForce 920M,Windows 10,2.2kg,720.32,0.0,0.0,500.0
1104,Lenovo,2 in 1 Convertible,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 6500U 2.5GHz,4GB,Intel HD Graphics 520,Windows 10,1.8kg,638.00,0.0,128.0,0.0
1105,Lenovo,Notebook,14.0,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2GB,Intel HD Graphics,Windows 10,1.5kg,229.00,64.0,0.0,0.0
1106,HP,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,6GB,AMD Radeon R5 M330,Windows 10,2.19kg,764.00,0.0,0.0,1024.0


In [ ]:
df['Weight'] = df['Weight'].apply(lambda x: float(x[:-2]))
df['Ram'] = df['Ram'].apply(lambda x: int(x[:-2]))
df

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Gpu,OpSys,Weight,Price_euros,Flash Storage,SSD,HDD
0,Apple,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,Intel HD Graphics 6000,macOS,1.34,898.94,128.0,0.0,0.0
1,Apple,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,AMD Radeon Pro 455,macOS,1.83,2537.45,0.0,512.0,0.0
2,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,Intel Iris Plus Graphics 650,macOS,1.37,1803.60,0.0,256.0,0.0
3,Acer,Notebook,15.6,1366x768,AMD A9-Series 9420 3GHz,4,AMD Radeon R5,Windows 10,2.10,400.00,0.0,0.0,500.0
4,Apple,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.2GHz,16,Intel Iris Pro Graphics,Mac OS X,2.04,2139.97,256.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,Asus,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,4,Nvidia GeForce 920M,Windows 10,2.20,720.32,0.0,0.0,500.0
1104,Lenovo,2 in 1 Convertible,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 6500U 2.5GHz,4,Intel HD Graphics 520,Windows 10,1.80,638.00,0.0,128.0,0.0
1105,Lenovo,Notebook,14.0,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2,Intel HD Graphics,Windows 10,1.50,229.00,64.0,0.0,0.0
1106,HP,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,6,AMD Radeon R5 M330,Windows 10,2.19,764.00,0.0,0.0,1024.0


In [ ]:
df['sx'] = df['ScreenResolution'].apply(lambda x: int(x.split()[-1].split('x')[0]))
df['sy'] = df['ScreenResolution'].apply(lambda x: int(x.split()[-1].split('x')[1]))
df = df.drop(columns=['ScreenResolution'])
df

,Company,TypeName,Inches,Cpu,Ram,Gpu,OpSys,Weight,Price_euros,Flash Storage,SSD,HDD,sx,sy
0,Apple,Ultrabook,13.3,Intel Core i5 1.8GHz,8,Intel HD Graphics 6000,macOS,1.34,898.94,128.0,0.0,0.0,1440,900
1,Apple,Ultrabook,15.4,Intel Core i7 2.7GHz,16,AMD Radeon Pro 455,macOS,1.83,2537.45,0.0,512.0,0.0,2880,1800
2,Apple,Ultrabook,13.3,Intel Core i5 3.1GHz,8,Intel Iris Plus Graphics 650,macOS,1.37,1803.60,0.0,256.0,0.0,2560,1600
3,Acer,Notebook,15.6,AMD A9-Series 9420 3GHz,4,AMD Radeon R5,Windows 10,2.10,400.00,0.0,0.0,500.0,1366,768
4,Apple,Ultrabook,15.4,Intel Core i7 2.2GHz,16,Intel Iris Pro Graphics,Mac OS X,2.04,2139.97,256.0,0.0,0.0,2880,1800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,Asus,Notebook,15.6,Intel Core i7 6500U 2.5GHz,4,Nvidia GeForce 920M,Windows 10,2.20,720.32,0.0,0.0,500.0,1366,768
1104,Lenovo,2 in 1 Convertible,14.0,Intel Core i7 6500U 2.5GHz,4,Intel HD Graphics 520,Windows 10,1.80,638.00,0.0,128.0,0.0,1920,1080
1105,Lenovo,Notebook,14.0,Intel Celeron Dual Core N3050 1.6GHz,2,Intel HD Graphics,Windows 10,1.50,229.00,64.0,0.0,0.0,1366,768
1106,HP,Notebook,15.6,Intel Core i7 6500U 2.5GHz,6,AMD Radeon R5 M330,Windows 10,2.19,764.00,0.0,0.0,1024.0,1366,768


In [ ]:
for col in df.select_dtypes(['object']).columns:
  print(col, len(df[col].value_counts()))

Company 18
TypeName 6
Cpu 112
Gpu 106
OpSys 9


In [ ]:
def cut(x):
  s = x.split()
  return s[0]

def cpu(x):
  s = x.split() 
  return float(s[-1][:-3])


df['CpuComp'] = df['Cpu'].apply(cut)
df['CpuGHz'] = df['Cpu'].apply(cpu)
df['GpuComp'] = df['Gpu'].apply(cut)
df = df.drop(columns=['Cpu', 'Gpu'])
df

,Company,TypeName,Inches,Ram,OpSys,Weight,Price_euros,Flash Storage,SSD,HDD,sx,sy,CpuComp,CpuGHz,GpuComp
0,Apple,Ultrabook,13.3,8,macOS,1.34,898.94,128.0,0.0,0.0,1440,900,Intel,1.8,Intel
1,Apple,Ultrabook,15.4,16,macOS,1.83,2537.45,0.0,512.0,0.0,2880,1800,Intel,2.7,AMD
2,Apple,Ultrabook,13.3,8,macOS,1.37,1803.60,0.0,256.0,0.0,2560,1600,Intel,3.1,Intel
3,Acer,Notebook,15.6,4,Windows 10,2.10,400.00,0.0,0.0,500.0,1366,768,AMD,3.0,AMD
4,Apple,Ultrabook,15.4,16,Mac OS X,2.04,2139.97,256.0,0.0,0.0,2880,1800,Intel,2.2,Intel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,Asus,Notebook,15.6,4,Windows 10,2.20,720.32,0.0,0.0,500.0,1366,768,Intel,2.5,Nvidia
1104,Lenovo,2 in 1 Convertible,14.0,4,Windows 10,1.80,638.00,0.0,128.0,0.0,1920,1080,Intel,2.5,Intel
1105,Lenovo,Notebook,14.0,2,Windows 10,1.50,229.00,64.0,0.0,0.0,1366,768,Intel,1.6,Intel
1106,HP,Notebook,15.6,6,Windows 10,2.19,764.00,0.0,0.0,1024.0,1366,768,Intel,2.5,AMD


In [ ]:
for col in df.select_dtypes(['object']).columns:
  print(col, len(df[col].value_counts()))

Company 18
TypeName 6
OpSys 9
CpuComp 3
GpuComp 4


In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

<ipython-input-22-935c2e534e54>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = df.corr()


,Inches,Ram,Weight,Price_euros,Flash Storage,SSD,HDD,sx,sy,CpuGHz
Inches,1.000000,0.238793,0.835675,0.049048,-0.228904,-0.118550,0.543101,-0.077391,-0.101785,0.285076
Ram,0.238793,1.000000,0.389375,0.727495,-0.055476,0.595877,0.116825,0.420047,0.412166,0.365554
Weight,0.835675,0.389375,1.000000,0.184584,-0.177632,-0.069859,0.526246,-0.048872,-0.068869,0.305136
Price_euros,0.049048,0.727495,0.184584,1.000000,-0.042930,0.669576,-0.098154,0.556244,0.552748,0.429402
Flash Storage,-0.228904,-0.055476,-0.177632,-0.042930,1.000000,-0.143715,-0.115336,0.004464,0.041634,-0.230788
SSD,-0.118550,0.595877,-0.069859,0.669576,-0.143715,1.000000,-0.385373,0.530173,0.519762,0.264940
HDD,0.543101,0.116825,0.526246,-0.098154,-0.115336,-0.385373,1.000000,-0.120128,-0.131878,0.125501
sx,-0.077391,0.420047,-0.048872,0.556244,0.004464,0.530173,-0.120128,1.000000,0.994193,0.178852
sy,-0.101785,0.412166,-0.068869,0.552748,0.041634,0.519762,-0.131878,0.994193,1.000000,0.163101
CpuGHz,0.285076,0.365554,0.305136,0.429402,-0.230788,0.264940,0.125501,0.178852,0.163101,1.000000


In [ ]:
df = df.drop(columns=['sy'])

In [ ]:
les = {}
for col in df.select_dtypes(['object']).columns:
  le = preprocessing.LabelEncoder()
  le.fit(df[col])
  df[col] = le.transform(df[col])
  les[col] = le 
df

,Company,TypeName,Inches,Ram,OpSys,Weight,Price_euros,Flash Storage,SSD,HDD,sx,CpuComp,CpuGHz,GpuComp
0,1,4,13.3,8,8,1.34,898.94,128.0,0.0,0.0,1440,1,1.8,2
1,1,4,15.4,16,8,1.83,2537.45,0.0,512.0,0.0,2880,1,2.7,0
2,1,4,13.3,8,8,1.37,1803.60,0.0,256.0,0.0,2560,1,3.1,2
3,0,3,15.6,4,5,2.10,400.00,0.0,0.0,500.0,1366,0,3.0,0
4,1,4,15.4,16,3,2.04,2139.97,256.0,0.0,0.0,2880,1,2.2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,2,3,15.6,4,5,2.20,720.32,0.0,0.0,500.0,1366,1,2.5,3
1104,9,0,14.0,4,5,1.80,638.00,0.0,128.0,0.0,1920,1,2.5,2
1105,9,3,14.0,2,5,1.50,229.00,64.0,0.0,0.0,1366,1,1.6,2
1106,7,3,15.6,6,5,2.19,764.00,0.0,0.0,1024.0,1366,1,2.5,0


In [ ]:
X, y = df.drop(columns=['Price_euros']), df['Price_euros']
X

,Company,TypeName,Inches,Ram,OpSys,Weight,Flash Storage,SSD,HDD,sx,CpuComp,CpuGHz,GpuComp
0,1,4,13.3,8,8,1.34,128.0,0.0,0.0,1440,1,1.8,2
1,1,4,15.4,16,8,1.83,0.0,512.0,0.0,2880,1,2.7,0
2,1,4,13.3,8,8,1.37,0.0,256.0,0.0,2560,1,3.1,2
3,0,3,15.6,4,5,2.10,0.0,0.0,500.0,1366,0,3.0,0
4,1,4,15.4,16,3,2.04,256.0,0.0,0.0,2880,1,2.2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,2,3,15.6,4,5,2.20,0.0,0.0,500.0,1366,1,2.5,3
1104,9,0,14.0,4,5,1.80,0.0,128.0,0.0,1920,1,2.5,2
1105,9,3,14.0,2,5,1.50,64.0,0.0,0.0,1366,1,1.6,2
1106,7,3,15.6,6,5,2.19,0.0,0.0,1024.0,1366,1,2.5,0


In [ ]:
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
X 

array([[-1.36223867,  1.13481587, -1.2212189 , ...,  0.21755989,
        -0.97634027, -0.02612031],
       [-1.36223867,  1.13481587,  0.25851455, ...,  0.21755989,
         0.79591566, -2.16992058],
       [-1.36223867,  1.13481587, -1.2212189 , ...,  0.21755989,
         1.58358496, -0.02612031],
       ...,
       [ 0.78326303,  0.32465093, -0.72797442, ...,  0.21755989,
        -1.37017492, -0.02612031],
       [ 0.24688761,  0.32465093,  0.39944154, ...,  0.21755989,
         0.40208101, -2.16992058],
       [-1.09405096,  0.32465093,  0.39944154, ...,  0.21755989,
        -1.37017492, -0.02612031]])

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
lr = LinearRegression()
cross_val_score(lr, X, y, cv=5, scoring='neg_mean_squared_error').mean()

-143388.210655117

In [ ]:
dtr = DecisionTreeRegressor()
cross_val_score(dtr, X, y, cv=5, scoring='neg_mean_squared_error').mean()

-175513.04134926197

In [ ]:
gbr = GradientBoostingRegressor(n_estimators=350)
cross_val_score(gbr, X, y, cv=5, scoring='neg_mean_squared_error').mean()

-88496.25405250286

#Test


In [ ]:
df_test = pd.read_csv("Price_euros_test.csv")
df_test

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
0,86,Acer,Aspire 3,Notebook,15.6,1366x768,AMD A9-Series 9420 3GHz,4GB,1TB HDD,AMD Radeon R5,Windows 10,2.1kg
1,1249,Razer,Blade Pro,Gaming,14.0,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,1TB SSD,Nvidia GeForce GTX 1060,Windows 10,1.95kg
2,122,Asus,VivoBook S15,Notebook,15.6,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8GB,256GB SSD,Nvidia GeForce 940MX,Windows 10,1.7kg
3,993,Asus,Rog GL753VE-DS74,Gaming,17.3,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1050 Ti,Windows 10,2.99kg
4,739,Dell,Inspiron 3567,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,12GB,1TB HDD,Intel HD Graphics 620,Windows 10,2.25kg
...,...,...,...,...,...,...,...,...,...,...,...,...
190,876,HP,Probook 450,Notebook,15.6,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,4GB,500GB HDD,Intel HD Graphics 520,Windows 10,2.07kg
191,282,Acer,Aspire A515-51G,Notebook,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8GB,1TB HDD,Nvidia GeForce MX130,Windows 10,2.2kg
192,528,HP,Spectre X360,Ultrabook,13.3,IPS Panel Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.32kg
193,1204,Dell,Inspiron 5578,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 7500U 2.7GHz,16GB,512GB SSD,Intel HD Graphics 620,Windows 10,2.09kg


In [ ]:
df_test.isna().sum()

laptop_ID            0
Company              0
Product              0
TypeName             0
Inches               0
ScreenResolution     2
Cpu                  0
Ram                  0
Memory               0
Gpu                  0
OpSys               12
Weight               4
dtype: int64

#Разрешение

In [ ]:
df_test['ScreenResolution'] = df_test['ScreenResolution'].apply(lambda x: "Full HD 1920x1080" if pd.isna(x) else x)

#OS

In [ ]:
indexes = df_test['OpSys'].isna()

In [ ]:
df_test.loc[indexes, 'OpSys'] = df_test.loc[indexes, 'Company'].apply(lambda x: 'macOS' if x == 'Apple' else 'Windows 10')
df_test[indexes]

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
27,447,Lenovo,IdeaPad 320-15AST,Notebook,17.3,1600x900,AMD A6-Series 9220 2.5GHz,8GB,1TB HDD,AMD Radeon R4,Windows 10,2.8kg
35,83,Apple,"MacBook 12""",Ultrabook,12.0,IPS Panel Retina Display 2304x1440,Intel Core i5 1.3GHz,8GB,512GB SSD,Intel HD Graphics 615,macOS,0.92kg
60,1107,Asus,Rog GL702VM-GC354T,Gaming,17.3,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1060,Windows 10,2.7kg
71,917,LG,Gram 15Z970,Ultrabook,15.6,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 7500U 2.7GHz,16GB,512GB SSD,Intel HD Graphics 620,Windows 10,1.08kg
74,498,Toshiba,Portege X30-D-10L,Ultrabook,13.3,Full HD / Touchscreen 1920x1080,Intel Core i7 7500U 2.7GHz,32GB,512GB SSD,Intel HD Graphics 620,Windows 10,1.05kg
84,1317,Lenovo,Yoga 900-13ISK,2 in 1 Convertible,13.3,IPS Panel Quad HD+ / Touchscreen 3200x1800,Intel Core i7 6500U 2.5GHz,16GB,512GB SSD,Intel HD Graphics 520,Windows 10,1.3kg
91,845,Asus,Rog GL702VS-BA023T,Gaming,17.3,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,2.9kg
104,624,HP,EliteBook 850,Ultrabook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,512GB SSD,Intel HD Graphics 620,Windows 10,1.84kg
122,200,Razer,Blade Pro,Gaming,17.3,4K Ultra HD / Touchscreen 3840x2160,Intel Core i7 7820HK 2.9GHz,32GB,1TB SSD,Nvidia GeForce GTX 1080,Windows 10,3.49kg
137,1096,Lenovo,IdeaPad Y900-17ISK,Gaming,17.3,IPS Panel Full HD 1920x1080,Intel Core i7 6820HK 2.7GHz,32GB,512GB SSD + 1.0TB Hybrid,Nvidia GeForce GTX 980M,Windows 10,4.6kg


#Вес

In [ ]:
indexes2 = df_test['Weight'].isna()
indexes1 = [not i for i in indexes2]

lr = LinearRegression()
lr.fit(df_test.loc[indexes1][['Inches']], df_test.loc[indexes1]['Weight'].apply(lambda x: float(x[:-2])))

LinearRegression()

In [ ]:
df_test.loc[indexes2, 'Weight'] = [str(i) + 'kg' for i in lr.predict(df_test.loc[indexes2][['Inches']])]
df_test[indexes2]

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
32,733,Lenovo,V310-15IKB (i5-7200U/4GB/1TB/No,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,4GB,1TB HDD,Intel HD Graphics 620,No OS,2.2917112512527633kg
92,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.4329807644702943kg
143,774,Acer,TravelMate P238-M,Notebook,13.3,1366x768,Intel Core i5 6200U 2.3GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 10,1.4329807644702943kg
152,344,Asus,X541NA (N3350/4GB/1TB/Linux),Notebook,15.6,1366x768,Intel Celeron Dual Core N3350 1.1GHz,4GB,1TB HDD,Intel HD Graphics 500,Linux,2.2917112512527633kg


#Обработка данных

In [ ]:
df_test['Company'].value_counts()

Lenovo       46
Dell         42
HP           42
Asus         23
Acer         12
MSI          10
Toshiba       7
Apple         4
Razer         3
Huawei        2
Mediacom      1
LG            1
Microsoft     1
Chuwi         1
Name: Company, dtype: int64

In [ ]:
df_test['Company'] = df_test['Company'].replace('Huawei', 'Lenovo')

In [ ]:
for col in df_test.select_dtypes(['object']).columns:
  print(col, len(df_test[col].value_counts()))

Company 14
Product 154
TypeName 6
ScreenResolution 23
Cpu 45
Ram 7
Memory 20
Gpu 51
OpSys 8
Weight 90


In [ ]:
df_test = df_test.drop(columns=['Product', 'laptop_ID'])

In [ ]:
T = 'Flash'
df_test['Flash Storage'] = df_test['Memory'].apply(memory)
T = 'SSD'
df_test['SSD'] = df_test['Memory'].apply(memory)
T = 'HDD'
df_test['HDD'] = df_test['Memory'].apply(memory)
df_test = df_test.drop(columns=['Memory'])
df_test

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Gpu,OpSys,Weight,Flash Storage,SSD,HDD
0,Acer,Notebook,15.6,1366x768,AMD A9-Series 9420 3GHz,4GB,AMD Radeon R5,Windows 10,2.1kg,0.0,0.0,1024.0
1,Razer,Gaming,14.0,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,Nvidia GeForce GTX 1060,Windows 10,1.95kg,0.0,1024.0,0.0
2,Asus,Notebook,15.6,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8GB,Nvidia GeForce 940MX,Windows 10,1.7kg,0.0,256.0,0.0
3,Asus,Gaming,17.3,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,Nvidia GeForce GTX 1050 Ti,Windows 10,2.99kg,0.0,256.0,1024.0
4,Dell,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,12GB,Intel HD Graphics 620,Windows 10,2.25kg,0.0,0.0,1024.0
...,...,...,...,...,...,...,...,...,...,...,...,...
190,HP,Notebook,15.6,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,4GB,Intel HD Graphics 520,Windows 10,2.07kg,0.0,0.0,500.0
191,Acer,Notebook,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8GB,Nvidia GeForce MX130,Windows 10,2.2kg,0.0,0.0,1024.0
192,HP,Ultrabook,13.3,IPS Panel Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,Intel HD Graphics 620,Windows 10,1.32kg,0.0,256.0,0.0
193,Dell,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 7500U 2.7GHz,16GB,Intel HD Graphics 620,Windows 10,2.09kg,0.0,512.0,0.0


In [ ]:
df_test['Weight'] = df_test['Weight'].apply(lambda x: float(x[:-2]))
df_test['Ram'] = df_test['Ram'].apply(lambda x: int(x[:-2]))
df_test

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Gpu,OpSys,Weight,Flash Storage,SSD,HDD
0,Acer,Notebook,15.6,1366x768,AMD A9-Series 9420 3GHz,4,AMD Radeon R5,Windows 10,2.10,0.0,0.0,1024.0
1,Razer,Gaming,14.0,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16,Nvidia GeForce GTX 1060,Windows 10,1.95,0.0,1024.0,0.0
2,Asus,Notebook,15.6,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8,Nvidia GeForce 940MX,Windows 10,1.70,0.0,256.0,0.0
3,Asus,Gaming,17.3,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16,Nvidia GeForce GTX 1050 Ti,Windows 10,2.99,0.0,256.0,1024.0
4,Dell,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,12,Intel HD Graphics 620,Windows 10,2.25,0.0,0.0,1024.0
...,...,...,...,...,...,...,...,...,...,...,...,...
190,HP,Notebook,15.6,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,4,Intel HD Graphics 520,Windows 10,2.07,0.0,0.0,500.0
191,Acer,Notebook,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8,Nvidia GeForce MX130,Windows 10,2.20,0.0,0.0,1024.0
192,HP,Ultrabook,13.3,IPS Panel Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8,Intel HD Graphics 620,Windows 10,1.32,0.0,256.0,0.0
193,Dell,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 7500U 2.7GHz,16,Intel HD Graphics 620,Windows 10,2.09,0.0,512.0,0.0


In [ ]:
df_test['sx'] = df_test['ScreenResolution'].apply(lambda x: int(x.split()[-1].split('x')[0]))
df_test['sy'] = df_test['ScreenResolution'].apply(lambda x: int(x.split()[-1].split('x')[1]))
df_test = df_test.drop(columns=['ScreenResolution'])
df_test

,Company,TypeName,Inches,Cpu,Ram,Gpu,OpSys,Weight,Flash Storage,SSD,HDD,sx,sy
0,Acer,Notebook,15.6,AMD A9-Series 9420 3GHz,4,AMD Radeon R5,Windows 10,2.10,0.0,0.0,1024.0,1366,768
1,Razer,Gaming,14.0,Intel Core i7 7700HQ 2.8GHz,16,Nvidia GeForce GTX 1060,Windows 10,1.95,0.0,1024.0,0.0,1920,1080
2,Asus,Notebook,15.6,Intel Core i7 8550U 1.8GHz,8,Nvidia GeForce 940MX,Windows 10,1.70,0.0,256.0,0.0,1920,1080
3,Asus,Gaming,17.3,Intel Core i7 7700HQ 2.8GHz,16,Nvidia GeForce GTX 1050 Ti,Windows 10,2.99,0.0,256.0,1024.0,1920,1080
4,Dell,Notebook,15.6,Intel Core i5 7200U 2.5GHz,12,Intel HD Graphics 620,Windows 10,2.25,0.0,0.0,1024.0,1366,768
...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,HP,Notebook,15.6,Intel Core i5 6200U 2.3GHz,4,Intel HD Graphics 520,Windows 10,2.07,0.0,0.0,500.0,1920,1080
191,Acer,Notebook,15.6,Intel Core i7 8550U 1.8GHz,8,Nvidia GeForce MX130,Windows 10,2.20,0.0,0.0,1024.0,1920,1080
192,HP,Ultrabook,13.3,Intel Core i7 7500U 2.7GHz,8,Intel HD Graphics 620,Windows 10,1.32,0.0,256.0,0.0,1920,1080
193,Dell,2 in 1 Convertible,15.6,Intel Core i7 7500U 2.7GHz,16,Intel HD Graphics 620,Windows 10,2.09,0.0,512.0,0.0,1920,1080


In [ ]:
for col in df_test.select_dtypes(['object']).columns:
  print(col, len(df_test[col].value_counts()))

Company 14
TypeName 6
Cpu 45
Gpu 51
OpSys 8


In [ ]:
df_test['CpuComp'] = df_test['Cpu'].apply(cut)
df_test['CpuGHz'] = df_test['Cpu'].apply(cpu)
df_test['GpuComp'] = df_test['Gpu'].apply(cut)
df_test = df_test.drop(columns=['Cpu', 'Gpu'])
df_test

,Company,TypeName,Inches,Ram,OpSys,Weight,Flash Storage,SSD,HDD,sx,sy,CpuComp,CpuGHz,GpuComp
0,Acer,Notebook,15.6,4,Windows 10,2.10,0.0,0.0,1024.0,1366,768,AMD,3.0,AMD
1,Razer,Gaming,14.0,16,Windows 10,1.95,0.0,1024.0,0.0,1920,1080,Intel,2.8,Nvidia
2,Asus,Notebook,15.6,8,Windows 10,1.70,0.0,256.0,0.0,1920,1080,Intel,1.8,Nvidia
3,Asus,Gaming,17.3,16,Windows 10,2.99,0.0,256.0,1024.0,1920,1080,Intel,2.8,Nvidia
4,Dell,Notebook,15.6,12,Windows 10,2.25,0.0,0.0,1024.0,1366,768,Intel,2.5,Intel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,HP,Notebook,15.6,4,Windows 10,2.07,0.0,0.0,500.0,1920,1080,Intel,2.3,Intel
191,Acer,Notebook,15.6,8,Windows 10,2.20,0.0,0.0,1024.0,1920,1080,Intel,1.8,Nvidia
192,HP,Ultrabook,13.3,8,Windows 10,1.32,0.0,256.0,0.0,1920,1080,Intel,2.7,Intel
193,Dell,2 in 1 Convertible,15.6,16,Windows 10,2.09,0.0,512.0,0.0,1920,1080,Intel,2.7,Intel


In [ ]:
df_test = df_test.drop(columns=['sy'])

In [ ]:
for col in df_test.select_dtypes(['object']).columns:
  print(col)
  df_test[col] = les[col].transform(df_test[col])
df_test

Company
TypeName
OpSys
CpuComp
GpuComp


,Company,TypeName,Inches,Ram,OpSys,Weight,Flash Storage,SSD,HDD,sx,CpuComp,CpuGHz,GpuComp
0,0,3,15.6,4,5,2.10,0.0,0.0,1024.0,1366,0,3.0,0
1,13,1,14.0,16,5,1.95,0.0,1024.0,0.0,1920,1,2.8,3
2,2,3,15.6,8,5,1.70,0.0,256.0,0.0,1920,1,1.8,3
3,2,1,17.3,16,5,2.99,0.0,256.0,1024.0,1920,1,2.8,3
4,4,3,15.6,12,5,2.25,0.0,0.0,1024.0,1366,1,2.5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,7,3,15.6,4,5,2.07,0.0,0.0,500.0,1920,1,2.3,2
191,0,3,15.6,8,5,2.20,0.0,0.0,1024.0,1920,1,1.8,3
192,7,4,13.3,8,5,1.32,0.0,256.0,0.0,1920,1,2.7,2
193,4,0,15.6,16,5,2.09,0.0,512.0,0.0,1920,1,2.7,2


In [ ]:
X_test = scaler.transform(df_test)
X_test

array([[-1.63042638,  0.32465093,  0.39944154, ..., -4.41813934,
         1.38666763, -2.16992058],
       [ 1.85601388, -1.29567894, -0.72797442, ...,  0.21755989,
         0.99283298,  1.04577983],
       [-1.09405096,  0.32465093,  0.39944154, ...,  0.21755989,
        -0.97634027,  1.04577983],
       ...,
       [ 0.24688761,  1.13481587, -1.2212189 , ...,  0.21755989,
         0.79591566, -0.02612031],
       [-0.55767553, -2.10584387,  0.39944154, ...,  0.21755989,
         0.79591566, -0.02612031],
       [ 0.24688761, -1.29567894,  1.597321  , ...,  0.21755989,
         0.99283298,  1.04577983]])

In [ ]:
gbr = GradientBoostingRegressor(n_estimators=350)
gbr.fit(X, y)
y_pred = gbr.predict(X_test)
y_pred

array([ 425.09808441, 3042.62620165,  806.80274631, 1643.3110365 ,
        679.15416598, 1251.97011374, 1758.3732572 ,  395.00132709,
        853.13652718,  870.80325477, 1152.74226614,  859.30133219,
       1054.64745349,  565.03607468, 1402.06891575, 2541.90325791,
       1070.15891314,  257.82480667, 1302.95343458,  657.58250056,
        756.29433339, 1240.15414916,  468.19609337, 1284.23939297,
       2032.9493473 , 1528.11283869,  727.59055957,  660.88244831,
        604.20761747, 3148.86600334,  777.34817615, 1473.66866445,
        452.5511472 , 1890.23610376,  819.47843339, 1511.59656398,
       1294.08511422,  324.43248916, 2013.33117626,  500.17988194,
       2592.13806035,  427.38639941,  306.59875229, 1154.14536594,
       1202.80032072, 1295.3031921 , 1777.30358746, 2716.49133097,
       1027.94987743, 1430.00021104, 1171.79158256,  807.75858014,
       1557.01841514, 2039.51667631,  482.80296422,  722.79854858,
       1460.14396387,  921.95175436,  654.25850418,  966.05992

In [ ]:
test_submission = df_test.copy()
test_submission['Price_euros'] = y_pred
test_submission = test_submission.drop(columns=list(set(test_submission.columns) - set(['Price_euros'])))
test_submission

,Price_euros
0,425.098084
1,3042.626202
2,806.802746
3,1643.311037
4,679.154166
...,...
190,700.840734
191,705.160478
192,1463.290402
193,1612.425458


In [ ]:
test_submission.to_csv('test_submission.csv', index=True)